In [ ]:
%cd ..

# ImbizoPM Agents Test

This notebook demonstrates how to use the ImbizoPM agents to create and execute a project planning workflow.

In [ ]:
from imbizopm_agents.graph import create_project_planning_graph, run_project_planning_graph
from IPython.display import display, Image
from langgraph.graph.graph import CompiledGraph

In [ ]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("ollama:cogito:32b")

# llm.invoke("Say 'hello'")

In [ ]:
# Create the project planning graph with checkpointing enabled
graph: CompiledGraph = create_project_planning_graph(llm, use_checkpointing=True, use_structured_output=False)

In [ ]:
try:
    display(Image(graph.get_graph().draw_png()))
except Exception as e:
    print(f"Error creating graph: {e}")

## Running the Project Planning Process

We run the project planning graph with a sample input. The system will process the input through all the agents in the workflow and may request human assistance when needed.

In [ ]:
PROJECT1 = """
Predict clinician responses to real-world clinical vignettes from rural Kenya, replicating expert nurse decision-making under low-resource healthcare constraints.
"""

PROJECT2 = """
AgentFlow is a Python library that automates the orchestration of multi-step agent workflows by integrating intelligent planning, routing, and execution of specialized operations.
"""

PROJECT = """
# Kenya-Clinical-Reasoning-Challenge

Zindi Challange : Predict clinician responses to real-world clinical vignettes from rural Kenya, replicating expert nurse decision-making under low-resource healthcare constraints.

## Can your model match real clinicians in rural Kenyan healthcare?

In many parts of the world, frontline healthcare workers make life-or-death decisions under pressure, with limited resources and specialist support. This challenge takes you to the heart of Kenyan healthcare, where nurses working across diverse counties and health facility levels face real-world clinical cases every day.

In this challenge, you'll be given 400 authentic clinical prompts—each one a carefully crafted vignette combining a nurse’s background and a complex medical situation. Your task is to predict the clinician’s response to each scenario, replicating the reasoning of trained professionals as closely as possible.

The vignettes span a wide range of medical domains, from maternal and child health to critical care, and were originally evaluated by expert clinicians and leading AI models (including GPT-4.0, LLAMA, and GEMINI). Each prompt includes details like the patient's presentation, nurse experience level, and facility type, simulating the nuance and challenge of real clinical environments in Kenya.

This dataset is small—only 400 training and 100 test samples—but that’s because collecting high-quality, expert-labelled medical data is hard. These are real-world cases, and every entry reflects the constraints and pressures faced by healthcare workers in underserved regions. In resource-limited settings, clinical decisions must be fast, accurate, and sensitive to both patient condition and system limitations. Your solution should aim to reflect that balance.

## About

The dataset contains real-world clinical vignettes drawn from frontline healthcare settings across Kenya. Each sample presents a prompt representing a clinical case scenario, along with the response from a human clinician. Your goal is to predict the clinician's response based on the prompt.

These vignettes simulate the types of decisions nurses in Kenya must make every day, particularly in low-resource environments where access to specialists or diagnostic equipment may be limited.

Each prompt was originally answered by expert clinicians as well as multiple large language models (LLMs) as part of a research initiative on AI in healthcare. For this challenge, we focus only on replicating the human clinician response.

## Important Notes

* These are real clinical scenarios, and the dataset is small because expert-labelled data is difficult and time-consuming to collect.
* Prompts are diverse across medical specialties, geographic regions, and healthcare facility levels, requiring broad clinical reasoning and adaptability.
* Responses may include abbreviations, structured reasoning (e.g. "Summary:", "Diagnosis:", "Plan:"), or free text.


## Evaluation

The evaluation metric for this challenge is the [ROUGE Score](https://zindi.africa/blog/zindi-error-metric-series-how-to-use-rouge-f-measure-for-machine-translation).

All clinician responses have been turned to lower case, punctuation removed and all paragraphs replaced with a space.

For every row in the dataset, submission files should contain 2 columns: ID and Translation.

Your submission file should look like this:

```
Master_Index     Clinician
```

```
ID_AAAitMaH      summary a 30 yr old...
```

If you are in the top 10 on the leaderboard you will be requested to prepare a [video](https://www.youtube.com/watch?v=oOaoWTd---E) to share with the host along with your code.

In the video you submit, you need to explain your approach to the problem as clearly as possible, including any relevant insights into the problem you discovered along the way (e.g. a clever way to engineer the raw features).

Final prizes will be judged and awarded by the host, based on the following criteria.

* The clarity of your pitch (how easy is it to understand the solution) - 25%
* The insights you obtained from tackling the problem - 15%
* How implementable is your code in a real application? Have you taken into account that the solution will be deployed on an edge device? - 25%
* Novel ideas taking into account complexities and real world applications - 25%
* Code that is clean, easy to read and work with - 10%

## Resource Restrictions

Your solution must be

* Quantized to reduce memory usage and improve inference speed
* Inference must be less than 100ms per vignette
* Inference RAM usage of less than 2 GB
* The maximum number of model parameters is 1 billion parameters
* Training should take no longer than 24 hours on a GPU similar to an NVIDIA T4 while inference should be on an NVIDIA Jetson Nano or equivalent.

Prizes

1st place: $5 000 USD

2nd place: $3 000 USD

3rd place: $2 000 USD

There are 5 000 Zindi points available. You can read more about [Zindi points here](https://zindi.africa/discussions/13959?utm_source=zindi&utm_medium=blog&utm_campaign=challenge_resources&utm_id=CR).

The results of this challenge will be written in a publication and challenge winners acknowledged as authors.

"""

In [ ]:
# Run the project planning process
results = []
for result in run_project_planning_graph(
    graph,
    user_input=PROJECT,
    thread_id="demo-run-1",  # Adding a unique thread ID for this run
    recursion_limit = 30
):
    results.append(result)

# Display the number of steps processed
print(f"Process completed with {len(results)} steps")